## ⛏️ DREBIN - Extraction

Notebook for extracting all the features used by DREBIN.

#### Imports

In [ ]:
# IMPORT
from   sklearn.feature_extraction.text  import CountVectorizer
from   tqdm                             import tqdm
import pandas                           as pd
import numpy                            as np
import os
import psutil

import drebinCommonModules as CM
import drebinGetApkData

In [ ]:
# Initialize TQDM library for Pandas
tqdm.pandas()

In [ ]:
print("⚡ START ⚡\n")

#### Parameters

In [ ]:
# Ground-Truth Dataset
#INPUT_PATH  = "../../../../0_Data/CSV/0_AndroCatSet.csv"
INPUT_PATH  = "../../../../0_Data/CSV/1_AndroCatSet_MiniTEST.csv"

# Output Path
OUTPUT_PATH = "../TMP/3a_DrebinData.csv"

TMP_PATH = "../TMP/"
if not os.path.exists(TMP_PATH):
    os.makedirs(TMP_PATH)
    print("📁🆕 Folder created       :", TMP_PATH)
else:
    print("📁✅ Folder already exists:", TMP_PATH)

In [ ]:
NUM_CPU = psutil.cpu_count()

### 1. Load Data

In [ ]:
appsDF = pd.read_csv(INPUT_PATH, index_col=False)
print("#️⃣ Apps: {}".format(appsDF.shape[0]))

appsDF.head(3)

### 2. Extraction

In [ ]:
print("Availaible CPU: {}".format(NUM_CPU))

In [ ]:
# Create folder for output
DREBIN_PATH = TMP_PATH + "Drebin/"
if not os.path.exists(DREBIN_PATH):
    os.makedirs(DREBIN_PATH)
    print("Folder created:", DREBIN_PATH)
else:
    print("Folder already exists:", DREBIN_PATH)

In [ ]:
# Extract Features
print("⛏️ Extract Data")
drebinGetApkData.GetApkData(NUM_CPU, DREBIN_PATH, appsDF['sha256'].to_list())

In [ ]:
# Create Feature Vectors
print("⛏️ 2. Create Feature Vectors")

dataFiles = CM.ListFiles(TMP_PATH + "Drebin/", ".data")

vectorizer = CountVectorizer(
                        input='filename',
                        tokenizer=lambda x: x.split('\n'),
                        token_pattern=None,
                        binary=True
)

featuresMatrix = vectorizer.fit_transform(dataFiles).toarray()
print("Shape of features Matrix: {}".format(np.array(featuresMatrix).shape))

In [ ]:
# Create a pandas DataFrame with a single column containing the array lists
appsDF['drebinFeatures'] = [arr.tolist() for arr in featuresMatrix]
  
print("📐 Binaries FV Length: {}".format(len(appsDF.loc[0,'drebinFeatures'])))

### 3. Save everything.

In [ ]:
appsDF.to_csv(OUTPUT_PATH, index=False)
appsDF.head(3)

In [ ]:
print("\n🔚 END \n")